Team members:

Geson Chong

Kalyan Madanapalli

Bret Tomko

Current Hypothesis:


Since the premise of this project was to solve a mystery, we thought that a person being fired may be a good enough reason to do such a thing (This may be part of the why). Due to this, we looked at the people that was fired between the months of May and November. We found that 36 people were not employed for the span between May and November. This list of people gave us a good starting point to look at. We then looked at what interactions these people may have in the other sections.

1) If an employee has been fired and is still communicating with people still within the company, that employee is more suspicious.

2) If a supervisor has a pattern of firing people, the supervisor can be classified as suspicious.

We thought a user accessing multiple computers could be suspicious as he might be accessing private information. We also figured connecting/disconnecting usb device at non-standard hours would be fishy since the suspect would be alone and could upload/download private information not meant for him.

3) If an employee has accessed multiple computers, then the employee is considered to be more suspicious than a user that hasn’t accessed as many computers. 

4) If an employee is using a usb device during non-standard work hours, then the employee is considered to be more suspicious.

Since there are many jobs that cause an employee to have continuous suspicious behavior, we thought it would be a good idea to find employees that are normally not suspicious but became suspicious for a short while before returning to normal activity.

5) If an employee is consistently suspicious throughout their whole employment then they are most likely not involved in this incident (same for someone who is consistently NOT suspicious).

6) If an employee has lots of changes from low suspicion to high suspicion this could be a good suspect to look at more closely.

7) If an employee who was mostly not suspicious from most of his employment becomes suspicious for a while before returning back to normal is likely involved in the incident.

Looking at the number of suspicious activities per day showed a huge spike on 08/16/2017, something probably occurred on this day or the days leading up to it.

What we plan to do next:

Create a list of employees that interesect all the hypotheses
    

In [ ]:
import pandas
import numpy
import matplotlib.pyplot
import datetime
%matplotlib inline
# Data files:
device = pandas.read_csv('device_info.csv')
email = pandas.read_csv('email_info.csv')
employee = pandas.read_csv('employee_info.csv')
http = pandas.read_csv('http_info.csv')
logon = pandas.read_csv('logon_info.csv')

Our first set of suspects we looked at were people that were fired and also at thier supervisors. We quickly saw that this on its own was not enough to figure out who to focus on. However we can compare future analytics against this list to help us validate our suspicions

In [ ]:
user_ids = numpy.unique(employee.user_id)
month = [""] * len(user_ids)
fired_date = pandas.DataFrame()
fired_date["user_id"] = user_ids
fired_date["month"] = month
fired_date = fired_date.set_index("user_id")

for i in range(0, len(employee)):
    currentRow = employee.iloc[i,:]
    fired_date.loc[currentRow.user_id].month = currentRow.month
    
people_fired = fired_date[fired_date.month != "11/1/17"].copy()

In [ ]:
names = []
supervisor = []
for i in people_fired.index.get_values():
    hold = employee[employee.user_id == i]
    names.append(hold.employee_name.iloc[0])
    supervisor.append(hold.supervisor.iloc[0])

people_fired["name"] = names
people_fired["supervisor"] = supervisor
people_fired

In [ ]:
supervisor_fire_count = pandas.DataFrame(people_fired.groupby("supervisor").count())
del supervisor_fire_count["name"]
supervisor_fire_count.rename(columns = {"month" : "Amount of people Fired"}, inplace=True)
supervisor_fire_count

The next set of data that we decided to analyze was activity that occured outside of the normal work day. And again by its self we could not see any clear path to continue our investigation from just this data but will be usefull in documenting all suspicious activities of the employes.

In [ ]:
strangeEarly = datetime.datetime.strptime("06:00:00", '%H:%M:%S')
strangeLate = datetime.datetime.strptime("18:00:00", '%H:%M:%S')

In [ ]:
email
emailOffHours = email
emailOffHours["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in emailOffHours.date]
emailOffHours = emailOffHours[emailOffHours.offHours]
emailOffHours["day"] = [x.split(" ")[0] for x in emailOffHours.date]
emailOffHours

In [ ]:
device
deviceOffHours = device
deviceOffHours["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in deviceOffHours.date]
deviceOffHours = deviceOffHours[deviceOffHours.offHours]
deviceOffHours["day"] = [x.split(" ")[0] for x in deviceOffHours.date]
deviceOffHours

In [ ]:
logon
logonOffHours = logon
logonOffHours["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in logonOffHours.date]
logonOffHours = logonOffHours[logonOffHours.offHours]
logonOffHours["day"] = [x.split(" ")[0] for x in logonOffHours.date]
logonOffHours

In [ ]:
http
httpOffHours = http
httpOffHours["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in httpOffHours.date]
httpOffHours = httpOffHours[httpOffHours.offHours]
httpOffHours["day"] = [x.split(" ")[0] for x in httpOffHours.date]
httpOffHours

In [ ]:
httpOffHours.iloc[:,:].groupby("user").day.count().plot.barh(figsize=(20,50))

Building off of the last idea we decided to look at every record and mark down if suspicious activites occured, for now it is just a 1 if it occured and a 0 if it did not occur but in the future we can change it to be < 1 if an event occured and the user's job make sense for them to be causing that event (e.x. itadmins will have smaller number for working outside of work hours). 

In [ ]:
susEmail = email.copy()
susEmail["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in susEmail.date]
susEmail["outside"] = [(not("@dtaa.com" in x)) for x in susEmail.to]
eToU = dict(zip(employee.email, employee.user_id))
uToE = dict(zip(employee.user_id, employee.email))
susEmail["user"] = [eToU[x] for x in susEmail.iloc[:,3]]

In [ ]:
susHttp = http.copy()
susHttp["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in susHttp.date]
susHttp["money"] = [ ("bank" in x) or ("pay" in x)  for x in susHttp.url]

In [ ]:
susLogon = logon.copy()
susLogon["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in susLogon.date]

In [ ]:
susDevice = device.copy()
susDevice["offHours"] = [(datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') < strangeEarly) or (datetime.datetime.strptime(x.split(" ")[1],'%H:%M:%S') > strangeLate)  for x in susDevice.date]

In [ ]:
susHttp["offHours"] = susHttp["offHours"].apply(float)
susHttp["money"] = susHttp["money"].apply(float)
susEmail["offHours"] = susEmail["offHours"].apply(float)
susEmail["outside"] = susEmail["outside"].apply(float)
susLogon["offHours"] = susLogon["offHours"].apply(float)
susDevice["offHours"] = susDevice["offHours"].apply(float)
susHttp["day"] = [x.split(" ")[0] for x in susHttp.date]
susEmail["day"] = [x.split(" ")[0] for x in susEmail.date]
susLogon["day"] = [x.split(" ")[0] for x in susLogon.date]
susDevice["day"] = [x.split(" ")[0] for x in susDevice.date]


Then we looked at the people with the most suspicious activity overall

In [ ]:
df = pandas.DataFrame(susLogon.groupby("user").offHours.sum())
df.rename(columns={"day": "logon"}, inplace=True)
df["http"] = susHttp.groupby("user").offHours.sum() + susHttp.groupby("user").money.sum()
df["email"] = susEmail.groupby("user").offHours.sum() + susEmail.groupby("user").outside.sum()
df["device"] = susDevice.groupby("user").offHours.sum()

df = df.fillna(0.0)
df["sum"] = df.sum(axis=1)
df = df.sort_values("sum", ascending=False)

Lastly for this analytics we decided to look at the top X suspicious people and look at thier mean suspicion for each day and the days prior. The idea here is to ignore people who are always supicious and to look for people who wernt suspicious who became suspicious then went back to not being suspicious, they might be involved in the incident and it could give us a timeframe for the incident also. 

In [ ]:
for suspect in df.index[0:20]:
    susEmail[susEmail.user == suspect].groupby("day").offHours.sum()
    suspect1 = pandas.DataFrame(susLogon[susLogon.user == suspect].groupby("day").offHours.sum())
    suspect1.rename(columns={"offHours": "logon"}, inplace=True)
    suspect1["http"] = susHttp[susHttp.user == suspect].groupby("day").offHours.sum() + susHttp[susHttp.user == suspect].groupby("day").money.sum()
    suspect1["email"] = susEmail[susEmail.user == suspect].groupby("day").offHours.sum() + susEmail[susEmail.user == suspect].groupby("day").outside.sum()
    suspect1["device"] = susDevice[susDevice.user == suspect].groupby("day").offHours.sum()

    suspect1 = suspect1.fillna(0.0)
    suspect1["total"] =suspect1.sum(axis=1)
    #suspect1 = suspect1.sort_values("sum", ascending=False)
    suspect1["consistency"] = [ suspect1.total.iloc[:i].mean()for i in range(len(suspect1))]
    suspect1.plot.line(x=suspect1.index, y="consistency", figsize=(15,5), title=suspect)

In [ ]:
susActivity = pandas.DataFrame(susLogon.groupby("day").offHours.sum())
susActivity.rename(columns={"offHours": "logon"}, inplace=True)
susActivity["http"] = susHttp.groupby("day").offHours.sum() + susHttp.groupby("day").money.sum()
susActivity["email"] = susEmail.groupby("day").offHours.sum() + susEmail.groupby("day").outside.sum()
susActivity["device"] = susDevice.groupby("day").offHours.sum()

susActivity = susActivity.fillna(0.0)
susActivity["total"] =susActivity.sum(axis=1)
#suspect1 = suspect1.sort_values("sum", ascending=False)
susActivity["consistency"] = [ susActivity.total.iloc[i-5:i].mean()for i in range(len(susActivity))]
susActivity.plot.line(x=susActivity.index, y="total", figsize=(15,5), title="All activity")


We also looked at how many computers were being used by each employee. We figured that an employee whose profession doesn't require to access many computers that is spotted using many computers could be flagged as suspicious.

In [ ]:
user_pcs = {}
device_user_pcs = {}
http_user_pcs = {}
logon_user_pcs = {}

for index, row in device.iterrows():
    user = row['user']
    if user in user_pcs:
        aset = user_pcs[user]
        aset.add(row['pc'])
        user_pcs[user] = aset    
        if user in device_user_pcs:
            aset2 = device_user_pcs[user]
            aset2.add(row['pc'])
            device_user_pcs[user] = aset2    
    else:
        user_pcs[user] = set([row['pc']])
        device_user_pcs[user] = set([row['pc']])

        
for index, row in http.iterrows():
    user = row['user']
    if user in user_pcs:
        aset = user_pcs[user]
        aset.add(row['pc'])
        user_pcs[user] = aset   
        if user in http_user_pcs:
            aset2 = http_user_pcs[user]
            aset2.add(row['pc'])
            http_user_pcs[user] = aset2     
    else:
        user_pcs[user] = set([row['pc']])
        http_user_pcs[user] = set([row['pc']])

for index, row in logon.iterrows():
    user = row['user']
    if user in user_pcs:
        aset = user_pcs[user]
        aset.add(row['pc'])
        user_pcs[user] = aset
        if user in logon_user_pcs:
            aset2 = logon_user_pcs[user]
            aset2.add(row['pc'])
            logon_user_pcs[user] = aset2
    else:
        user_pcs[user] = set([row['pc']])
        logon_user_pcs[user] = set([row['pc']])

user_pcs

We decided to run k-means with 5 clusters on all the employees based on their suspicous activity parameters. We figured that a suspicious group would arise from one of these clusters.

In [ ]:
import sklearn.cluster
import sklearn.manifold

df2 = pandas.DataFrame(df.iloc[:,0:4])


km = sklearn.cluster.KMeans(n_clusters=5)
labels = km.fit_predict(df2)
df2["cluster"] = labels
df2 # ran kmeans with 5 clusters

We used MDS to flatten the parameters into 2 dimensions in order to visualize the data in a scatter plot.

In [ ]:
mds = sklearn.manifold.MDS(n_components=2, dissimilarity='euclidean', max_iter = 1000)
data2D = mds.fit_transform(df2.copy())
df3 = pandas.DataFrame(data2D, columns=["x", "y"])
df3.index = df2.index
df3["cluster"] = labels

plt  = df3.plot.scatter(x = "x", y = "y", c = "cluster", colormap = matplotlib.pyplot.cm.Paired, sharex = False)
plt